In [2]:
import pandas as pd
import numpy as np

import nltk
# nltk.download('stopwords')
# nltk.download('punkt_tab')

from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import json


# Data Modeling 
# pick two topics to compare
# develop a way to identify which topic a post is talking about 
# check occurences of different topics of conversation over time
# could these different conversation topics appear during a game period more often?


In [3]:


ps = PorterStemmer()

def text2tokens(text):
	stop_words = set(stopwords.words('english'))
	text = str(text)
	text = text.lower()
	textList = word_tokenize(text)
	textList = [word for word in textList if word not in stop_words and len(word) >= 3]

	textList = [ps.stem(word) for word in textList]
	return textList
    
def gen_bow(df, column):
	df['tokens'] = df[column].apply(text2tokens)
	dct = Dictionary(df['tokens'])
	dct.filter_extremes(no_below=5, no_above=0.5)
	df['bow'] = df['tokens'].apply(dct.doc2bow)
	worddict = {}

	for i in range(len(df)):
		for token in df.at[i,'bow']:
			if token in list(worddict.keys()):
				worddict[token] += 1
			else: 
				worddict[token] = 1

	df.drop('tokens', axis=1, inplace=True)	
	return df, worddict



In [4]:


posNegWords = pd.read_excel('posNegList.xlsx').dropna()
# print(posNegWords.dtypes)

posWords = [ps.stem(word) for word in posNegWords['Positive Sense Word List'].to_list()]
negWords = [ps.stem(word) for word in posNegWords['Negative Sense Word List'].to_list()]



def get_sentiment(list):
	sentiment = 0
	
	for word in list:
		if word in posWords:
			sentiment += 1
		elif word in negWords:
			sentiment -= 1
	
	return sentiment
			
rdf = pd.read_csv('hw2_step1_reddit_posts.csv')

rdf, word_dict = gen_bow(rdf, 'text')
rdf['tokens'] = rdf['text'].apply(text2tokens)
# rdf['sentiment'] = rdf['tokens'].apply(get_sentiment)

sent_array = np.array(rdf['tokens'].apply(get_sentiment))
max_sent = sent_array.max()
min_sent = sent_array.min()

rdf['sentiment'] = (sent_array - min_sent) / (max_sent - min_sent)

display(rdf)


,text,time_of_post,game,team,bow,tokens,sentiment
0,first career complete game no-hitter for Means!,2021-05-05,"['401227501', '401227486']",BAL,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]","[first, career, complet, game, no-hitt, mean]",0.275093
1,#FIRST NO HITTER IN 30 YEARS LETS GOOOOOOOO,2021-05-05,"['401227501', '401227486']",BAL,"[(2, 1), (6, 1), (7, 1), (8, 1), (9, 1)]","[first, hitter, year, let, goooooooo]",0.260223
2,I can't believe the son of a bitch actually di...,2021-05-05,"['401227501', '401227486']",BAL,"[(7, 1), (10, 1), (11, 1), (12, 1), (13, 1), (...","[n't, believ, son, bitch, actual, never, thoug...",0.260223
3,EVERYBODY GET THE FUCK IN HERE PAT HIT HIS FIR...,2021-05-05,"['401227501', '401227486']",BAL,"[(2, 1), (4, 1), (7, 1), (20, 1), (21, 1), (22...","[everybodi, get, fuck, pat, hit, first, home, ...",0.271375
4,WE'VE SUCKED FOR SO LONG HOW DO I TAKE THIS IN...,2021-05-05,"['401227501', '401227486']",BAL,"[(28, 1), (29, 1), (30, 1), (31, 1), (32, 1), ...","['ve, suck, long, take, without, tear]",0.252788
...,...,...,...,...,...,...,...
6625,I feel like I'm taking fucking crazy pills. I'...,2017-07-02,['370702114'],TOR,"[(15, 1), (22, 1), (31, 1), (83, 2), (128, 1),...","[feel, like, take, fuck, crazi, pill, 'll, ask...",0.263941
6626,"Anyone else here high,and thought it was gonna...",2017-07-03,['370703110'],TOR,"[(19, 1), (20, 1), (220, 1), (511, 1), (567, 1...","[anyon, els, high, thought, gon, arrow, refer,...",0.263941
6627,It's crazy.\r\nAt the end of the season the th...,2017-07-03,['370703110'],TOR,"[(19, 1), (23, 2), (137, 1), (149, 1), (225, 1...","[crazi, end, season, thought, even, joke, laug...",0.263941
6628,"Welp, never thought I'd see a headline like th...",2017-07-03,['370703110'],TOR,"[(15, 1), (17, 1), (19, 1), (83, 1), (600, 1)]","[welp, never, thought, see, headlin, like, lif...",0.260223


In [5]:
display(rdf.sort_values(by='sentiment'))

,text,time_of_post,game,team,bow,tokens,sentiment
3183,SHIT PISS AND CUM SHIT PISS AND CUM SHIT PISS ...,2019-10-20,[],BOS,"[(53, 69), (476, 69)]","[shit, piss, cum, shit, piss, cum, shit, piss,...",0.000000
3166,"Congrats, Houston!\r\n\r\nFUCK THE YANKEES\r\n...",2019-10-20,[],BOS,"[(22, 11), (82, 1), (248, 11), (320, 1), (369,...","[congrat, houston, fuck, yanke, fuck, yanke, f...",0.152416
1487,FUCK CHRIS SALE AND HIS GOOFY ASS DELIVERY\r\n...,2019-09-10,['401076896'],NYY,"[(12, 1), (22, 8), (34, 2), (58, 1), (81, 1), ...","[fuck, chri, sale, goofi, ass, deliveri, fuck,...",0.226766
912,"Fuck you Kluber, Fuck you Bauer, Fuck you Jose...",2017-10-11,[],NYY,"[(17, 1), (22, 5), (166, 1), (240, 1), (660, 1...","[fuck, kluber, fuck, bauer, fuck, jose, ramire...",0.234201
2459,Fuck this fucking bullshit\r\n\r\nDavid price ...,2018-09-19,['380919110'],NYY,"[(8, 1), (22, 6), (37, 1), (68, 1), (176, 1), ...","[fuck, fuck, bullshit, david, price, domin, ya...",0.234201
...,...,...,...,...,...,...,...
2742,David Ortiz changed my life and I'll always ca...,2016-10-12,[],BOS,"[(2, 2), (3, 7), (13, 2), (14, 4), (15, 2), (1...","[david, ortiz, chang, life, 'll, alway, carri,...",0.639405
5946,Is...this post in the right place? Have I mis...,2016-04-26,['360426114'],TOR,"[(8, 1), (9, 3), (13, 1), (14, 3), (20, 5), (2...","[..., post, right, place, miss, someth, know, ...",0.639405
2478,Gary Sanchez has had one of the most up and do...,2021-06-24,['401228171'],NYY,"[(0, 4), (2, 5), (3, 7), (9, 4), (14, 1), (19,...","[gari, sanchez, one, career, athlet, last, thi...",0.776952
3291,"Excuse the throwaway (for anonymity), but I pr...",2018-10-19,['381018118'],BOS,"[(0, 2), (2, 1), (3, 7), (9, 8), (14, 11), (15...","[excus, throwaway, anonym, promis, say, true, ...",0.914498
